In [29]:
from part2_house_value_regression import Regressor, save_regressor
from pandas import read_csv

def train_regressor(model, model_name, hidden_count: int, nb_epochs=100, learning_rate=0.01):
    output_label = "median_house_value"

    # Use pandas to read CSV data as it contains various object types
    # Feel free to use another CSV reader tool
    # But remember that LabTS tests take Pandas DataFrame as inputs
    data = read_csv("housing.csv")

    # Splitting input and output
    x_train = data.loc[:, data.columns != output_label]
    y_train = data.loc[:, [output_label]]

    eval_data = read_csv("housing_eval.csv")
    x_eval = eval_data.loc[:, eval_data.columns != output_label]
    y_eval = eval_data.loc[:, [output_label]]

    # Training
    # This example trains on the whole available dataset.
    # You probably want to separate some held-out data
    # to make sure the model isn't overfitting
    regressor = Regressor(x_train, model=model, nb_epoch=nb_epochs, batch_size=32, learning_rate=learning_rate)
    regressor.hidden_count = hidden_count
    regressor.fit(x_train, y_train, x_eval=x_eval, y_eval=y_eval)

    # Error
    error = regressor.score(x_train, y_train)
    print("\nRegressor error: {}\n".format(error))

    save_regressor(regressor, model_name)

    return regressor, model_name

In [30]:
from pandas import DataFrame
from math import log10
from pathlib import Path

def dump_history(regressor: Regressor, model_name: str):
    history_train = regressor.loss_history()
    history_eval = regressor.loss_history_eval()

    history_out_path = Path(f'assets/{model_name}-lr-{int(-log10(regressor.learning_rate))}-epch-{regressor.nb_epoch}.csv')  
    history_out_path.parent.mkdir(parents=True, exist_ok=True) 

    history = DataFrame(data={'train': history_train, 'eval': history_eval})
    history.to_csv(history_out_path)

In [31]:
from matplotlib.pyplot import figure, plot

def get_eval_all_fig_name(count: int, lr: float):
    return f'eval-hyperparam-search-{count}-lr-1e-{int(-log10(lr))}'

def output_eval_all_fig(count: int, lr: float, output_fs: bool = False, output_ui: bool = False):
    fig_eval_all = figure(get_eval_all_fig_name(count, lr))
    fig_eval_all.legend()

    if output_ui: fig_eval_all.show()
    if output_fs: fig_eval_all.savefig(f'assets/eval-hyperparam-search-{count}-lr-1e-{int(-log10(lr))}.png')

def display_loss_history(regressor: Regressor, model_name: str):
    loss_history = regressor.loss_history()
    loss_history_eval = regressor.loss_history_eval()

    fig = figure()
    plot(loss_history, label='training')
    plot(loss_history_eval, label='eval')
    fig.show()
    fig.savefig(f'assets/{model_name}-lr-{int(-log10(regressor.learning_rate))}-epch-{regressor.nb_epoch}.png')

    figure(get_eval_all_fig_name(regressor.hidden_count, regressor.learning_rate))
    plot(loss_history_eval, label=f'{model_name}-lr-{int(-log10(regressor.learning_rate))}-epch-{regressor.nb_epoch}')
    output_eval_all_fig(regressor.hidden_count, regressor.learning_rate, output_fs=True)


In [32]:
from multiprocessing import Pool
from torch.nn import Module, Sequential, Linear, ReLU, SiLU

def output(regressor: Regressor, model_name: str):
    dump_history(regressor, model_name)
    display_loss_history(regressor, model_name)

def hyperparam_search(count: int, size: int, activation_func: Module, lr: float, name: str):
    layers = \
        [Linear(13, size), activation_func()] + \
        [Linear(size, size), activation_func()] * count + \
        [Linear(size, 1)]

    output(*train_regressor(
        Sequential(*layers),
        model_name=name,
        hidden_count=count,
        nb_epochs=100,
        learning_rate=lr,
    ))

    print(f'Completed {name}!')

for count in range(1, 5):
    for size in [5, 10, 50, 100, 200, 500]:
        for lr in [0.001, 0.0001, 0.00001]:
            hyperparam_search(count, size, ReLU, lr, f'relu-{count}-{size}')
            # hyperparam_search(count, size, SiLU, lr, f'silu-{count}-{size}')



Epoch 0 error (train): 104647.8359375
Epoch 0 error (eval): 103274.4609375
Epoch 1 error (train): 72731.8125
Epoch 1 error (eval): 72344.7109375
Epoch 2 error (train): 67622.40625
Epoch 2 error (eval): 67492.890625
Epoch 3 error (train): 65958.375
Epoch 3 error (eval): 66039.0546875
Epoch 4 error (train): 64976.59765625
Epoch 4 error (eval): 65082.7890625
Epoch 5 error (train): 64224.6875
Epoch 5 error (eval): 64548.5703125
Epoch 6 error (train): 63661.5546875
Epoch 6 error (eval): 64122.27734375
Epoch 7 error (train): 63441.84375
Epoch 7 error (eval): 64005.71484375
Epoch 8 error (train): 62760.38671875
Epoch 8 error (eval): 63417.125
Epoch 9 error (train): 62461.25
Epoch 9 error (eval): 63142.98046875
Epoch 10 error (train): 62284.328125
Epoch 10 error (eval): 62959.12109375
Epoch 11 error (train): 61840.453125
Epoch 11 error (eval): 62725.01171875
Epoch 12 error (train): 61347.6875
Epoch 12 error (eval): 62275.22265625
Epoch 13 error (train): 61095.64453125
Epoch 13 error (eval): 62

In [ ]:
# from torch.nn import Sequential, Linear, ReLU

# display_loss_history(*train_regressor(
#     Sequential(
#         Linear(13, 1000),
#         ReLU(),
#         Linear(1000, 500),
#         ReLU(),
#         Linear(500, 500),
#         ReLU(),
#         Linear(500, 200),
#         ReLU(),
#         Linear(200, 1),
#     ),
#     model_name='relu1000x500x500x200',
#     nb_epochs=100,
# ))
# from torch.nn import Sequential, Linear, SiLU

# display_loss_history(*train_regressor(
#     Sequential(
#         Linear(13, 1000),
#         SiLU(),
#         Linear(1000, 500),
#         SiLU(),
#         Linear(500, 500),
#         SiLU(),
#         Linear(500, 200),
#         SiLU(),
#         Linear(200, 1),
#     ),
#     model_name='silu1000x500x500x200',
#     nb_epochs=100,
# ))

Epoch 0 error (train): 0.22046521306037903
Epoch 0 error (eval): 0.22405286133289337


KeyboardInterrupt: 